# 6. 프로젝트 : 멋진 작사가 만들기

## 6.1 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Well, summer slipped us underneath her tongue', 'Our days and nights are perfumed with obsession', 'Half of my wardrobe is on your bedroom floor']


## 6.2 데이터 정제

In [4]:
import re
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [10]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> well , summer slipped us underneath her tongue <end>',
 '<start> our days and nights are perfumed with obsession <end>',
 '<start> half of my wardrobe is on your bedroom floor <end>',
 '<start> use our eyes , throw our hands overboard i am your sweetheart psychopathic crush <end>',
 '<start> drink up your movements , still i can t get enough <end>',
 '<start> i overthink your p punctuation use <end>',
 '<start> not my fault , just a thing that my mind do a rush at the beginning <end>',
 '<start> i get caught up , just for a minute <end>',
 '<start> but lover , you re the one to blame , all that you re doing <end>',
 '<start> can you hear the violence ? <end>']

In [31]:
import tensorflow as tf
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 136   4 ...   0   0   0]
 [  2 155 377 ...   0   0   0]
 [  2 541  19 ...   0   0   0]
 ...
 [  2   5  22 ...   0   0   0]
 [  2   5  22 ...   0   0   0]
 [  2   5  22 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fab702a4410>


In [32]:
print(tensor[:3, :15])

[[   2  136    4  584 3121  131 1299   69  969    3    0    0    0    0
     0]
 [   2  155  377    8  858   77 9560   31 8541    3    0    0    0    0
     0]
 [   2  541   19   13 5339   26   18   21 1473  363    3    0    0    0
     0]]


In [33]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2  136    4  584 3121  131 1299   69  969    3    0    0    0    0
    0    0    0    0    0]
[ 136    4  584 3121  131 1299   69  969    3    0    0    0    0    0
    0    0    0    0    0]


In [34]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

## 6.3 평가 데이터셋 분리

In [35]:
from sklearn.model_selection import train_test_split


enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                                tgt_input, 
                                                                test_size=0.2,
                                                                shuffle=True)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)



Source Train: (140599, 19)
Target Train: (140599, 19)


1. 학습데이터 갯수가 140599로 권장 데이터인 124960개를 초과하였으나 권장사항이기 때문에 일단 진행

In [ ]:
tensor, tokenizer